In [ ]:
getwd()
setwd('..')
getwd()

In [ ]:
library(Matrix)
library(Seurat)
library(mclust)
library(SingleCellExperiment)
library(stringr)
#library(clustree)
#citation("mclust")


In [ ]:
#cc.genes$s.genes

In [ ]:
#cc.genes$g2m.genes

In [ ]:
#cc.genes

In [ ]:
read_in_data <-function(save_name){
    exp_data=readMM(sprintf("datasets/extract/%s.data.counts.mm",save_name))
 
    exp_data_row=read.table(sprintf('datasets/extract/%s.data.row',save_name))$V1
    exp_data_col=read.table(sprintf('datasets/extract/%s.data.col',save_name))$V1    
    rownames(exp_data)=exp_data_row
    colnames(exp_data)=exp_data_col    

    metadatarow=read.table(sprintf('datasets/extract/%s.metadatarow.tsv',save_name),sep='\t')
    metadatacol=read.table(sprintf('datasets/extract/%s.metadatacol.tsv',save_name),sep='\t')  
    
    metadatacol['size_factor']=read.table(sprintf('datasets/extract/%s.size_factor.tsv',save_name),sep='\t')$V1

    sce <- SingleCellExperiment(list(counts=as.matrix(exp_data)),rowData=metadatarow,colData=metadatacol)
    sce
}  

calculate_low_dim <-function(sce, pca_dim=10){
    counts = assay(sce, "counts")
    seurat <- CreateSeuratObject(counts = counts, project = "scRNAseq", assay = "RNA",
                                         min.cells = 0, min.features = 0)
    
    seurat <- NormalizeData(seurat); seurat <- ScaleData(seurat,features = rownames(seurat)); seurat <- FindVariableFeatures(seurat, selection.method = "vst", nfeatures = 2000)
    seurat <- RunPCA(seurat, features = VariableFeatures(object = seurat)); seurat <- RunTSNE(seurat, dims= 1:pca_dim)
    reducedDims(sce) <- list(PCA=Embeddings(seurat, reduction = "pca")[,1:pca_dim], TSNE=Embeddings(seurat, reduction = "tsne"))
    sce
}

In [ ]:
save_deg<-function(sce,save_name,method){
    
    seurat_object <- CreateSeuratObject(counts = counts(sce), project = "scRNAseq", assay = "RNA",
                                         min.cells = 0, min.features = 0,
                                         meta.data = as.data.frame(colData(sce)))
    
    Idents(seurat_object)=colData(sce)$phenoid
    
    marker_found=FindAllMarkers(seurat_object, test.use=method, logfc.threshold = 0.25, min.pct = 0.1)
    
    write.table(marker_found,file=sprintf('datasets/extract/%s.deg.%s.tsv',save_name,method),sep='\t')
}

In [ ]:
dataset_name_original_all=list(
'Kohinbulk_filtered',
'HumanLiver_filtered',
'Zhengmix8eq_filtered')

length(dataset_name_original_all)

In [ ]:
dataset_name_tabula_all=list("TabulaAorta_filtered",
"TabulaBladder_filtered",
"TabulaBrainMyeloid_filtered",
"TabulaBrainNonMyeloid_filtered",
"TabulaDiaphragm_filtered",
"TabulaFat_filtered",
"TabulaHeart_filtered",
"TabulaKidney_filtered",
"TabulaLargeIntestine_filtered",
"TabulaLimbMuscle_filtered",
"TabulaLiver_filtered",
"TabulaLung_filtered",
"TabulaMammaryGland_filtered",
"TabulaMarrow_filtered",
"TabulaPancreas_filtered",
"TabulaSkin_filtered",
"TabulaSpleen_filtered",
"TabulaThymus_filtered",
"TabulaTongue_filtered",
"TabulaTrachea_filtered")

In [ ]:
dataset_name_simul_all=list()
for (ncells_total in c('1000','2000','5000','10000')){
    for (prop in c('1e-2','5e-3','1e-3','5e-4')){
        for(i in c(1:10)){
            dataset_name<-sprintf('Simul_%s_%s_%s_filtered',ncells_total,prop,i)
            dataset_name_simul_all<-append(dataset_name_simul_all,dataset_name)
        }

    }
}

In [ ]:
dataset_name_all=append(dataset_name_original_all,dataset_name_tabula_all)
length(dataset_name_all)

In [ ]:
for(method in c("wilcox", "bimod", "roc", "t")){
    for(dataset_name in dataset_name_all[1:3]){
        start_time=as.numeric(Sys.time())

        sce=read_in_data(dataset_name);seurat=as.Seurat(sce, data=NULL)
        #seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

        #seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

        #seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))    
        Idents(seurat)=seurat@meta.data['phenoid']


        marker_found=FindAllMarkers(seurat, test.use=method, logfc.threshold = 0.25, min.pct = 0.1)
        write.table(marker_found,file=sprintf('datasets/extract/%s.realdeg.%s.tsv',dataset_name,method),sep='\t')    
        
        rm(list=c('seurat'))
        
    }    
}

In [ ]:
for(dataset_name in dataset_name_all[1:length(dataset_name_all)]){
    for (res in c(0.5,1.0,1.5)){
        print(dataset_name)
        print(res)
        
        start_time=as.numeric(Sys.time())
        res_str=str_replace(sprintf('%.1f',res),fixed('.'),'_')

        seurat=read_in_data(dataset_name)
        
        seurat_=calculate_low_dim(seurat, pca_dim=10)
    
        seurat=as.Seurat(seurat, data=NULL)
        seurat[['pca']] <- CreateDimReducObject(embeddings = reducedDim(seurat_), assay='RNA')
        seurat <- FindNeighbors(seurat, dims = 1:10)
        seurat <- FindClusters(seurat, resolution = res)   

        write.table(seurat@meta.data['seurat_clusters'],
                    sprintf('datasets/extract/%s.clustering.%s.label.tsv',dataset_name,res_str),sep='\t')
        
        end_time=as.numeric(Sys.time())
        
        time_preprocessing=end_time-start_time
        
        Idents(seurat)=seurat@meta.data['seurat_clusters']
        
        start_time=as.numeric(Sys.time())
        markers <- FindAllMarkers(seurat, test.use="wilcox")#, only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
        write.table(markers,
                    sprintf('datasets/extract/%s.clustering.%s.deg.tsv',dataset_name,res_str),sep='\t')    
        end_time=as.numeric(Sys.time())
        time_deg=end_time-start_time
        
        write.table(time_preprocessing+time_deg,
                    file=sprintf('datasets/extract/%s.clustering.%s.deg.runtime.tsv',dataset_name,res_str),
                    sep='\t',
                    row.names=FALSE, 
                    col.names=FALSE)          
        
        start_time=as.numeric(Sys.time())
        markers_all <- FindAllMarkers(seurat, test.use="wilcox",logfc.threshold =0)#, only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
        write.table(markers_all,
                    sprintf('datasets/extract/%s.clustering.%s.deg.all.tsv',dataset_name,res_str),sep='\t')  
        end_time=as.numeric(Sys.time())
        time_deg_all=end_time-start_time
        
        write.table(time_preprocessing+time_deg_all,
                    file=sprintf('datasets/extract/%s.clustering.%s.deg_all.runtime.tsv',dataset_name,res_str),
                    sep='\t',
                    row.names=FALSE, 
                    col.names=FALSE)
        
        

    }
    write.table(Embeddings(seurat[['tsne']]),
                file=sprintf('datasets/extract/%s.tsne.vst.2000.tsv',dataset_name),
                sep='\t',
                row.names=TRUE,
                col.names=TRUE
               )     
    
}

# DEG specific

# Koh

In [ ]:
lfc_thres=0.25

In [ ]:
res=1.0

In [ ]:
dataset_name='Kohinbulk_filtered'

sce=read_in_data(dataset_name);seurat=as.Seurat(sce, data=NULL)

seurat_hvg <- NormalizeData(seurat)

all.genes <- rownames(seurat_hvg)
seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))


seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = res)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)    

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")   

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")  

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='APS',ident.2='MPS',group.by='phenoid',logfc.threshold=lfc_thres)    

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.APS.MPS.tsv',dataset_name),sep='\t')    


markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='MPS',ident.2='APS',group.by='phenoid',logfc.threshold=lfc_thres)    

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.MPS.APS.tsv',dataset_name),sep='\t') 

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='APS',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.APS.tsv',dataset_name),sep='\t')

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='MPS',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.MPS.tsv',dataset_name),sep='\t')

In [ ]:
FeaturePlot(seurat_hvg_pca_label_cluster_tsne,features=c('ENSG00000156574'))

# Lung

In [ ]:
dataset_name='TabulaLung_filtered'

sce=read_in_data(dataset_name);seurat=as.Seurat(sce, data=NULL)

seurat_hvg <- NormalizeData(seurat)

all.genes <- rownames(seurat_hvg)
seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))


seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = res)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)    

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")   

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")  

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='T cell',ident.2='natural killer cell',group.by='phenoid',logfc.threshold=lfc_thres)     

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.T_cell.natural_killer_cell.tsv',dataset_name),sep='\t')

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='natural killer cell',ident.2='T cell',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.natural_killer_cell.T_cell.tsv',dataset_name),sep='\t') 

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='natural killer cell',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.natural_killer_cell.tsv',dataset_name),sep='\t') 

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='T cell',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.T_cell.tsv',dataset_name),sep='\t') 

# Liver

In [ ]:
dataset_name='HumanLiver_filtered'

sce=read_in_data(dataset_name);seurat=as.Seurat(sce, data=NULL)

seurat_hvg <- NormalizeData(seurat)

all.genes <- rownames(seurat_hvg)
seurat_hvg=ScaleData(seurat_hvg,features = all.genes)

seurat_hvg <- FindVariableFeatures(seurat_hvg, selection.method = "vst", nfeatures = 2000)

seurat_hvg_pca <- RunPCA(seurat_hvg, features = VariableFeatures(object = seurat_hvg))


seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = res)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)    

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")   

DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")  

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='gd T cells',ident.2='ab T cells',group.by='phenoid',logfc.threshold=lfc_thres)    

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.gd_T_cells.ab_T_cells.tsv',dataset_name),sep='\t')    

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='gd T cells',ident.2='NK cells',group.by='phenoid',logfc.threshold=lfc_thres)    

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.gd_T_cells.NK_cells.tsv',dataset_name),sep='\t') 

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='gd T cells',group.by='phenoid',logfc.threshold=lfc_thres)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.gd_T_cells.tsv',dataset_name),sep='\t') 

markers=FindMarkers(seurat_hvg_pca_label_cluster, ident.1='NK cells',group.by='phenoid',logfc.threshold=0)  

write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.NK_cells.tsv',dataset_name),sep='\t') 

In [ ]:
FeaturePlot(seurat_hvg_pca_label_cluster_tsne,features=c('GNLY'))

In [ ]:
'datasets/extract/%s.clustering.%s.deg.tsv',dataset_name,res_str),sep='\t')    

In [ ]:
markers <- FindAllMarkers(seurat_hvg_pca_label_cluster, test.use="wilcox")#, only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
write.table(markers,
            sprintf('datasets/extract/%s.truelabel.deg.tsv',dataset_name,res_str),sep='\t')    

In [ ]:
write.table(markers,
            sprintf('datasets/extract/%s.clustering.%s.deg.tsv',dataset_name,res_str),sep='\t')    

In [ ]:
FindAllMarkers(seurat_hvg_pca_label_cluster, test.use="wilcox")

In [ ]:
head(FindMarkers(seurat_hvg_pca_label_cluster, ident.1='MPS',ident.2='APS',group.by='phenoid'))

In [ ]:
help(FeaturePlot)

In [ ]:
help(FindMarkers)

In [ ]:
FeaturePlot(seurat_hvg_pca_label_cluster_tsne,features='ENSG00000125691')

In [ ]:
slotNames(seurat_hvg_pca_label_cluster[['RNA']])

In [ ]:
FindMarkers(seurat_hvg_pca_label_cluster, logfc.threshold=1, ident.1='APS',ident.2='MPS',group.by='phenoid')

In [ ]:
help(FindMarkers)

In [ ]:
FindMarkers(seurat_hvg_pca_label_cluster, test.use='LR',logfc.threshold=1, ident.1='APS',ident.2='MPS',group.by='phenoid')

In [ ]:
help(FindMarkers)

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")    
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")    

In [ ]:
help(str_replace)

In [ ]:
Idents(seurat_hvg_pca_label_cluster)=seurat_hvg_pca_label_cluster@meta.data['seurat_clusters']
markers <- FindAllMarkers(seurat_hvg_pca_label_cluster, test.use="wilcox")#, only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
#
#write.table(markers,sprintf('datasets/extract/%s.clusteringdeg.%s.tsv',save_name,'0_5'),sep='\t')

In [ ]:
markers

In [ ]:
help(FindAllMarkers)

In [ ]:
seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = 0.5)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")

In [ ]:
#FeaturePlot(seurat_hvg_pca_label_cluster_tsne,'MKI67')

In [ ]:
write.table(seurat_hvg_pca_label_cluster_tsne@meta.data['seurat_clusters'],sprintf('datasets/extract/%s.clusterlabel.%s.tsv',save_name,'0_5'),sep='\t')


In [ ]:
seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = 1.0)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")

In [ ]:
write.table(seurat_hvg_pca_label_cluster_tsne@meta.data['seurat_clusters'],sprintf('datasets/extract/%s.clusterlabel.%s.tsv',save_name,'1_0'),sep='\t')

In [ ]:
seurat_hvg_pca_label <- FindNeighbors(seurat_hvg_pca, dims = 1:10)
seurat_hvg_pca_label_cluster <- FindClusters(seurat_hvg_pca_label, resolution = 1.5)
seurat_hvg_pca_label_cluster_tsne <- RunTSNE(seurat_hvg_pca_label_cluster, dims= 1:10)

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='phenoid', reduction = "tsne")

In [ ]:
DimPlot(seurat_hvg_pca_label_cluster_tsne, group.by='seurat_clusters', reduction = "tsne")

In [ ]:
write.table(seurat_hvg_pca_label_cluster_tsne@meta.data['seurat_clusters'],sprintf('datasets/extract/%s.clusterlabel.%s.tsv',save_name,'1_5'),sep='\t')

In [ ]:
#Idents(seurat)=seurat@meta.data['seurat_clusters']
#markers <- FindAllMarkers(seurat, test.use="wilcox", only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
#write.table(markers,sprintf('datasets/extract/%s.clusteringdeg.%s.tsv',save_name,'0_5'),sep='\t')
#write.table(seurat_hvg_pca_label_cluster@meta.data['seurat_clusters'],sprintf('datasets/extract/%s.clusterlabel.%s.tsv',save_name,0.1),sep='\t')
#DimPlot(seurat_hvg_pca_label_cluster, reduction = "tsne")

In [ ]:
priunt(1)

In [ ]:
seurat1 <- FindNeighbors(seurat1, dims = 1:10)
seurat1 <- FindClusters(seurat1, resolution = 0.5)

Idents(seurat1)=seurat1@meta.data['seurat_clusters']
markers <- FindAllMarkers(seurat1, test.use="wilcox", only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
write.table(markers,sprintf('datasets/extract/%s.clusteringdeg.%s.tsv',save_name,'0_5'),sep='\t')

DimPlot(seurat1, reduction = "tsne")

In [ ]:
seurat1 <- FindNeighbors(seurat1, dims = 1:10)
seurat1 <- FindClusters(seurat1, resolution = 1.0)

Idents(seurat1)=seurat1@meta.data['seurat_clusters']
markers <- FindAllMarkers(seurat1, test.use="wilcox", only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
write.table(markers,sprintf('datasets/extract/%s.clusteringdeg.%s.tsv',save_name,'1_0'),sep='\t')

DimPlot(seurat1, reduction = "tsne")

In [ ]:
seurat1 <- FindNeighbors(seurat1, dims = 1:10)
seurat1 <- FindClusters(seurat1, resolution = 1.5)

Idents(seurat1)=seurat1@meta.data['seurat_clusters']
markers <- FindAllMarkers(seurat1, test.use="wilcox", only.pos = TRUE, min.pct = 0.1, logfc.threshold = 0.25)
write.table(markers,sprintf('datasets/extract/%s.clusteringdeg.%s.tsv',save_name,'1_5'),sep='\t')

DimPlot(seurat1, reduction = "tsne")